# Source Catalog Loading, Plotting and Exploring Examples

In [ ]:
import numpy as np
import os.path as op

from astropy.table import Table, unique

from hetdex_tools.source_catalog import plot_source_group
from hetdex_api.elixer_widget_cls import ElixerWidget
from hetdex_api.query_widget import QueryWidget
from hetdex_api.amp_widget import AmpWidget
from hetdex_api.config import HDRconfig

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

This catalog is a combination of sources from the continuum source catalog, the curated detection line catalog of the same version and the confirmed AGN detection IDs from Chenxu Liu's AGN search. Each row is for a detection and its unique catalog detectid. Detectid's of AGN sources *may* appear twice if they are also found in the continuum or curated line catalogs. This is intentional as information about the detection type is used to guess the redshift of the sources. The column `'det_type'` can be `['line','cont','agn']`. Version 2.1.3 includes a column called "source_type" where we make a preliminary guess whether the sources is 'agn','lae', 'oii', 'star', 'lzg'(low-z galaxy without line emission), 'unsure'. 

In [ ]:
# Enter the catalog version

version = '3.0.0'

config = HDRconfig()
catfile = op.join(config.hdr_dir['hdr3'], 'catalogs', 'source_catalog_' + version + '.fits')
source_table = Table.read(catfile)

print('Source catalog was found at {}'.format(catfile))

Columns are as follows, most are directly from the detection and elixer catalogs. Check the wiki for more info.
    
    https://luna.mpe.mpg.de/wikihetdex/index.php/Column_Info

In [ ]:
source_table.columns

You can sort the table using the `unique` function from `astropy.table` on the `source_id` key. For nearby galaxies, stars and AGN, many detectids can comprise the same source (ie. the same `source_id`). When you reduce the catalog using the `unique` function, only 1 detectid remains for the source. It is sorted such that the source with a wavelength matching the `z_guess` value is selected and the detectid is located close to the positional center of the source group. 

In [ ]:
# selecting for S/N>5.5 line sources and all continuum sources
sel_sn = (source_table['sn'] > 5.5) & (source_table['det_type'] == 'line')
sel_cont = source_table['det_type'] == 'cont'
sel_cat = sel_sn | sel_cont
uniq_table = unique(source_table[sel_cat], keys='source_id')

In [ ]:
len(uniq_table)

In [ ]:
star = uniq_table['source_id'][uniq_table['source_type'] == 'star']
oii = uniq_table['source_id'][uniq_table['source_type'] == 'oii']
lae = uniq_table['source_id'][uniq_table['source_type'] == 'lae']
agn = uniq_table['source_id'][uniq_table['source_type'] == 'agn']
lzg = uniq_table['source_id'][uniq_table['source_type'] == 'lzg']

print('There are {} low-z gals, {} stars, {} OII emitters, {} AGN, and {} LAEs'. \
      format(np.size(lzg), np.size(star), np.size(oii), np.size(agn), np.size(lae)))

## Here is an example of plotting a group

In [ ]:
%matplotlib inline

In [ ]:
# the source_ids are sorted by member number ('n_member' in the catalog)
# This is how many detections are found in the source_id group

# Here's a query for high S/N LAEs
sel = (uniq_table['source_type'] == 'lae') & (uniq_table['sn'] > 6.5)
sid = uniq_table['source_id'][sel][20000]

# Index a star 
#sid = star[100]

# Index an OII galaxy
#sid = oii[0]

# Index an LAE
#sid = lae[0]

# Index the AGN list:
#sid = agn[0]

# This step grabs the source group so we can look at each source in Elixer
sel_group = (source_table['source_id'] == sid) 
group = source_table[sel_group]

#print(sid)
#print(group)

# this function overplots the source group over an image
# use label=True to see detectid's on image
# use save=True to save the image to you working directory
# the size of the line emission is scaled to the S/N of the line

plot_source_group(source_id=sid, source_table=source_table, label=False, save=False)
ew = ElixerWidget(detectlist = source_table['detectid'][sel_group])

## Follow a detection in the 2D image with AmpWidget 

This will map the detectid from the ElixerWidget above to the amp widget. The detection is marked with a circle.

In [ ]:
aw = AmpWidget(detectid=ew.detectbox.value)

## Follow up with QueryWidget

You can extract fibers on demand around the detection via QueryWiget:

In [ ]:
qw = QueryWidget(detectid=ew.detectbox.value)

## Exploring fun with plotly 

In [ ]:
import plotly.express as px

In [ ]:
df = source_table[source_table['sn']>5.5].to_pandas()

In [ ]:
df['marker_size'] = 3
fig = px.scatter(df, 
                 template='plotly', 
                 x='sn', y='z_hetdex', 
                 color='source_type', 
                 size='marker_size', 
                 size_max=3, 
                 marginal_y='histogram', 
                 marginal_x='histogram',
                 hover_data=['detectid','source_id'])
fig.show()